Получение исходных данных

In [ ]:
from basepolygon_class import GoalPolygon

place = GoalPolygon('Жилой комплекс «Одинцово-1»')
v = place.polygon

Геометрическое упрощение

In [ ]:
def get_xy(s):
    x, y = s.centroid.coords.xy
    return x[0], y[0]

In [ ]:
from shapely.affinity import translate

s = v.simplify(10, preserve_topology=True)
x, y = get_xy(s)
st = translate(s, 0-x, 0-y)

st

In [ ]:
# убрать бы эту ебань
coor_list = (list(st.boundary.coords[0:len(st.boundary.coords) - 1]))
print (coor_list)

Декомпозиция периметра

In [ ]:
import numpy as np
from math import pi, cos, sin

target = pi/2
basis = np.array([[1, 0, 0],[0, 1, 0]])
tolerance=0.75

In [ ]:
transform = np.array(
    [
        [cos(target), -sin(target), 0],
        [sin(target), cos(target), 0], on('Жилой комплекс «Одинцово-1»')
        v = place.polygon
Геометрическое
упрощение


def get_xy(s):
    x, y = s.centroid.coords.xy
    return x[0], y[0]


from shapely.affinity import translate

s = v.simplify(10, preserve_topology=True)
x, y = get_xy(s)
st = translate(s, 0 - x, 0 - y)

st
# убрать бы эту ебань
coor_list = (list(st.boundary.coords[0:len(st.boundary.coords) - 1]))
print(coor_list)
Декомпозиция
периметра
import numpy as np
from math import pi, cos, sin

target = pi / 2
basis = np.array([[1, 0, 0], [0, 1, 0]])
tolerance = 0.75
transform = np.array(
    [
        [cos(target), -sin(target), 0],
        [sin(target), cos(target), 0],
        [0, 0, 1]
    ]
)
transform

In [ ]:
metric = np.complex(np.cross(basis[0], basis[1])[-1],
                             np.cross(basis[1], basis[0])[-1])


In [ ]:
from itertools import islice, cycle

def vec_search(point_list, key):
    cr = []
    n = list(islice(cycle(range(len(point_list))), 0, len(point_list) + 2))
    for i in range(len(point_list)):
        v_one = [point_list[n[i]][0] - point_list[n[i + 1]][0], point_list[n[i]][1] - point_list[n[i + 1]][1], 0]
        v_two = [point_list[n[i + 1]][0] - point_list[n[i + 2]][0], point_list[n[i + 1]][1] - point_list[n[i + 2]][1],0]
        v_o_norm, v_t_norm = v_one / np.linalg.norm(v_one), v_two / np.linalg.norm(v_two)
        if np.cross(v_o_norm, v_t_norm)[2] < key:
            cr.append(n[i + 1])
        else:
            pass
    cross = [*cr, cr[0] + cr[-1] + 1]
    double_points = [*range(len(point_list)), *range(len(point_list))]
    segments = [double_points[cross[c]:cross[c + 1]+1] for c in range(len(cross) - 1)]
    return segments

In [ ]:
bb = vec_search(coor_list, 0.7)
print(bb)

In [ ]:
def chains (point_list):
    key_val = vec_search(point_list, 0.7)
    vectors =[]
    for i in key_val:
        temp =[]
        for l in range(len(i)-1):
            vec = [point_list[i[l]][0] - point_list[i[l+1]][0], point_list[i[l]][1] - point_list[i[l+1]][1]]
            temp.append(vec)
        vectors.append(temp)
    return vectors



In [ ]:
res_vec = chains(coor_list)
print(res_vec)

SVD
Декомпозиция

In [ ]:
# Это техническая часть для работы с локальными данными
import json

f = open("/Users/andrewastakhov/dumps/pl.json", "r")
points = json.load(f)['pts']

In [ ]:
from scipy.linalg import svd
import numpy as np

A = np.asarray(res_vec)

In [ ]:
print(A)

Пример с одной матрицей

In [ ]:
a = np.asarray(A[0])

In [ ]:
a

In [ ]:
U, s, Vh=svd(a)

In [ ]:
s

Получить Sigma из s Хуй знает зачем сайпи говорят что это полезно

In [ ]:
sigma = np.zeros(a.shape)
for i in range(min(a.shape)):
            sigma[i, i] = s[i]
sigma

А точно, как минимум для обратной проверки, "B" должно быть равно "a"

In [ ]:
B = U @ sigma @ Vh

In [ ]:
a

In [ ]:
B

Теперь на торче
Класс треннируется на основе отсортированых данных выпуклой декомпозиции периметра


In [ ]:
import torch
from polygon_division import vector_search, points_to_point_chain

In [ ]:
class SvdItem:
    keys=["A", "U", "S", "Vh", "Sigma"]
    def __init__(self, m, collection=None, **kwargs):
        self.A = torch.tensor(m)
        self.collection=collection
        self.U = None
        self.S = None
        self.Vh = None
        self.Sigma = None
        self.svd(**kwargs)
        if collection is not None:
            self.id = f'{collection.id}:{len(collection.matrices)}'
            print(f'set item id: {self.id}')
            collection.A.append(self.A)



    def svd(self, **kwargs):
        U, S, Vh = torch.linalg.svd(self.A, **kwargs)
        Sigma = torch.diag(S)
        self.U, self.S, self.Vh, self.Sigma=U, S, Vh, Sigma
        print(self.U, self.S, self.Vh, self.Sigma)
        if self.collection:
            for v, k in zip([A, U, S, Vh, Sigma], self.__class__.keys):
                (self.collection.__dict__[k]).append(v)

        return U, S, Vh, Sigma
    def pack(self):
        self.collection.__dict__[self.__class__.__name__][self.collection.id]|={self.id:self}
        return self.id, self



class SvdPack:
    def __init__(self):
        self._matrices = []
        self.id = f'{0}:{0}'
        self.A, self.U, self.S, self.Vh, self.Sigma = [], [], [], [], []
        self.tree= {self.id:{}}



    @property
    def matrices(self):
        return self._matrices
    @matrices.setter
    def matrices(self, val, **kwargs):
        if len(val.shape) == 2:
            self._matrices.append(SvdItem(val, collection=self, **kwargs))
        else:
            for v in val:
                self._matrices.append(SvdItem(v, collection=self, **kwargs))


In [ ]:
pck = SvdPack()

In [ ]:
a_=np.asarray(res_vec, dtype=object)

In [ ]:
len(a_.shape)

In [ ]:
pck.matrices = a_

In [ ]:
pck.matrices

In [ ]:
pck.S

In [ ]:
pck.Vh

In [ ]:
def chains_ (point_list):
    vectors =[]
    for i in point_list:
        temp =[]
        for l in range(len(i)-1):
            vec = [i[l][0] - i[l+1][0], i[l][1] - i[l+1][1]]
            temp.append(vec)
        vectors.append(temp)
    return vectors

chains_(aa)


In [ ]:
def transform_chains_vh(chains, t):
    chains_t=[]
    for i, clust in enumerate(chains):
        _t=[]
        for j in clust:
            _ = t.Vh[i] @ np.asarray(coor_list[j])
            _t.append(np.asarray(_).tolist())
        chains_t.append(_t)
    return chains_t

aa = transform_chains_vh(bb, pck)
aa

In [ ]:
def angle(a, b):
    return np.arccos(a @ b)

for i, v_ in enumerate(aa):
    va = np.asarray(v_)
    vb = np.asarray(res_vec[i])
    print(angle(va[1], vb[1]))




